<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [2]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def get_outliers(data):
    """
    Detecta valores atípicos en un conjunto de datos utilizando el método del rango intercuartílico (IQR).

    Args:
        data (Series): Serie de pandas que contiene los datos numéricos para analizar.

    Returns:
        Series: Serie de pandas que contiene los valores atípicos detectados.
    """
    q1,q3 = data.quantile([0.25,0.75])
    iqr = q3-q1
    upper_fence = q3 + (1.5*iqr)
    lower_fence = q1 - (1.5*iqr)
    outliers = data[(data < lower_fence) | (data > upper_fence)]
    # print(iqr)
    # print(lower_fence, upper_fence)
    return outliers

def decimalpoint(string_num):
    return float(string_num.replace('.', '').replace(',','.'))

In [4]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

code_mapper = dict()
for k,v in cod_comunidades.items():
    code_mapper[v] = k

## Demanda y generación energética

In [5]:
def extraer_df(ruta):
    years = os.listdir(ruta)
    df_final = pd.DataFrame()
    for year in years:
        ruta_year = os.path.join(ruta, year)
        files = os.listdir(ruta_year)
        df_com = pd.DataFrame()
        for file in files:
            ruta_file = os.path.join(ruta_year, file)
            df_archivo = pd.read_csv(ruta_file, index_col=0)
            df_archivo["code_com"] = cod_comunidades[file.split('.')[0]]
            df_com = pd.concat([df_com, df_archivo])
        df_final = pd.concat([df_final,df_com])
        df_final.reset_index(drop=True, inplace=True)
    df_final["mes"] = df_final["datetime"].apply(lambda x: pd.to_datetime(x).month)
    df_final["year"] = df_final["datetime"].apply(lambda x: pd.to_datetime(x).year)
    df_final.drop(columns='datetime', inplace=True)
    return df_final

In [6]:
df_demanda = extraer_df(ruta='..\datos\demanda')
df_generacion = extraer_df(ruta='..\datos\generacion')

Vemos que la columna de magnitude no nos da información relevante.

In [7]:
df_generacion.sample(5)

,value,percentage,type,magnitude,code_com,mes,year
2825,4233.212,0.006295,Solar térmica,NaN,9,12,2021
1424,74.397,0.001649,Solar fotovoltaica,NaN,6,12,2020
666,3463.736,0.014658,Otras renovables,NaN,15,8,2019
3549,283132.874,1.000000,Generación renovable,NaN,21,5,2021
365,763644.114,0.707553,Eólica,NaN,8,6,2019


Está entera vacía

In [8]:
df_generacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3557 entries, 0 to 3556
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   value       3557 non-null   float64
 1   percentage  3557 non-null   float64
 2   type        3557 non-null   object 
 3   magnitude   0 non-null      float64
 4   code_com    3557 non-null   int64  
 5   mes         3557 non-null   int64  
 6   year        3557 non-null   int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 194.7+ KB


In [9]:
df_generacion.drop(columns='magnitude', inplace=True)

Podemos comprobar en la web de la [REE](https://www.ree.es/es/datos/generacion/estructura-generacion), que los datos energéticos están recogidos en MWh.

In [10]:
df_generacion.rename(columns={"value" : "energia (MWh)"}, inplace=True)

In [11]:
df_generacion.sample(10)

,energia (MWh),percentage,type,code_com,mes,year
586,1.186684e+04,0.043244,Solar fotovoltaica,14,11,2019
22,7.642175e+05,0.729591,Eólica,4,11,2019
1367,8.216830e+02,0.006774,Otras renovables,8742,3,2020
1858,2.667179e+05,1.000000,Generación renovable,15,2,2020
1246,9.775764e+05,1.000000,Generación renovable,4,2,2020
2741,1.513642e+06,0.716957,Eólica,8,12,2021
2194,3.238950e+03,0.042235,Solar fotovoltaica,10,2,2020
4,8.853545e+04,0.068497,Hidráulica,4,5,2019
762,3.360691e+05,0.297249,Hidráulica,17,10,2019
536,7.633675e+03,0.227990,Residuos renovables,13,9,2019


Vamos a ver dónde encontramos los outliers:

In [12]:
df_generacion.iloc[get_outliers(df_generacion["energia (MWh)"]).index]['code_com'].map(code_mapper).value_counts()

code_com
Castilla y León               96
Galicia                       91
Andalucía                     71
Castilla - La Mancha          68
Aragón                        64
Cataluña                      39
Extremadura                   27
Principado de Asturias         6
Comunitat Valenciana           4
Comunidad Foral de Navarra     3
Name: count, dtype: int64

Sobretodo en Castilla y León y Galicia. En cuanto a las fechas donde se concentran estos valores atípicos:

In [13]:
df_generacion.iloc[get_outliers(df_generacion["energia (MWh)"]).index]['year'].value_counts().head()

year
2021    173
2020    156
2019    140
Name: count, dtype: int64

Además observamos errores en el cálculo de porcentajes:

In [14]:
df_generacion[(df_generacion["mes"] == 12)&(df_generacion["year"] == 2021)&(df_generacion["code_com"] == 8745)]

,energia (MWh),percentage,type,code_com,mes,year
3328,3.6600,1.0,Solar fotovoltaica,8745,12,2021
3340,536.5155,1.0,Residuos renovables,8745,12,2021
3351,540.1755,1.0,Generación renovable,8745,12,2021


Definimos una función y aplicamos para calcular nuestros propios porcentajes.

In [15]:
def calcular_porcentaje(row):
    total = df_generacion[(df_generacion["mes"] == row["mes"]) &
                          (df_generacion["year"] == row["year"]) &
                          (df_generacion["code_com"] == row["code_com"]) &
                          (df_generacion["type"] == 'Generación renovable')]["energia (MWh)"].values[0]
    return round(row["energia (MWh)"] / total, 5)

df_generacion["porcentaje_generacion"] = df_generacion.apply(calcular_porcentaje, axis=1)

In [16]:
df_generacion.drop(columns='percentage', inplace=True)

In [17]:
df_generacion[(df_generacion["mes"] == 12)&(df_generacion["year"] == 2021)&(df_generacion["code_com"] == 8745)]

,energia (MWh),type,code_com,mes,year,porcentaje_generacion
3328,3.6600,Solar fotovoltaica,8745,12,2021,0.00678
3340,536.5155,Residuos renovables,8745,12,2021,0.99322
3351,540.1755,Generación renovable,8745,12,2021,1.00000


Ahora, para evitar distorsiones a la hora del análisis, eliminaremos los valores que representan los totales, es decir los que son de tipo: 'Generación renovable'.

In [18]:
df_generacion.drop(index = df_generacion[df_generacion["type"] == "Generación renovable"].index, inplace=True)

In [19]:
df_generacion.head()

,energia (MWh),type,code_com,mes,year,porcentaje_generacion
0,25288.605,Hidráulica,4,1,2019,0.02952
1,33933.350,Hidráulica,4,2,2019,0.03078
2,35913.000,Hidráulica,4,3,2019,0.02964
3,44050.848,Hidráulica,4,4,2019,0.04134
4,88535.452,Hidráulica,4,5,2019,0.06850


## INE: info demográfica y PIB

In [20]:
df_demog = pd.read_csv('..\datos\INE\datos_demograficos.csv', encoding='latin-1', sep=';')
provincias = df_demog["Provincias"].unique()

Primero voy a hacer un diccionario para mapear las diferentes provincias a los códigos.

In [21]:
import re

cods = dict()
for provincia in provincias:
    splitted = provincia.split()
    cods["".join(re.findall(pattern= r" (\D+)", string=provincia))] = splitted[0]

cod_provincias = dict()
for k,v in cods.items():
    try:
        cod_provincias[k] = int(v)
    except:
        cod_provincias[k] = v

In [22]:
df_demog["Provincias"] = df_demog["Provincias"].apply(lambda x:"".join(re.findall(pattern= r" (\D+)", string=x))).map(cod_provincias)

In [23]:
df_demog["Total"] = df_demog["Total"].apply(decimalpoint)

In [24]:
df_demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7632 entries, 0 to 7631
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Provincias               7632 non-null   object 
 1   Edad (3 grupos de edad)  7632 non-null   object 
 2   Españoles/Extranjeros    7632 non-null   object 
 3   Sexo                     7632 non-null   object 
 4   Año                      7632 non-null   int64  
 5   Total                    7632 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 357.9+ KB


Tenemos un montón de datos de los totales que podrían distorsionar el análisis. Vamos a eliminarlos, ya los calcularemos más tarde igualmente.

In [25]:
df_demog.head()

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2021,47385107.0
1,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2020,47450795.0
2,TOTAL,TOTAL EDADES,TOTAL,Ambos sexos,2019,47026208.0
3,TOTAL,TOTAL EDADES,TOTAL,Hombres,2021,23222953.0
4,TOTAL,TOTAL EDADES,TOTAL,Hombres,2020,23255590.0


In [26]:
a_drop = []
for row, index in zip(df_demog.values, df_demog.index):
    if any('total' in str(ele).lower() for ele in row) or any('ambos' in str(ele).lower() for ele in row):
        a_drop.append(index)

In [27]:
df_demog.drop(index = a_drop, inplace=True)
df_demog.reset_index(drop=True, inplace=True)

In [28]:
df_demog.head()

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,2,0-15 años,Españoles,Hombres,2021,27304.0
1,2,0-15 años,Españoles,Hombres,2020,27979.0
2,2,0-15 años,Españoles,Hombres,2019,28513.0
3,2,0-15 años,Españoles,Mujeres,2021,25547.0
4,2,0-15 años,Españoles,Mujeres,2020,25953.0


Por último eliminamos los valores de % Extranjeros, ya que es una métrica y nos distorsiona los datos. Esta métrica la podremos obtener más adelante.

In [29]:
df_demog.drop(index = df_demog[df_demog["Españoles/Extranjeros"] == '% Extranjeros'].index).reset_index(inplace=True)

In [30]:
df_demog.head()

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,2,0-15 años,Españoles,Hombres,2021,27304.0
1,2,0-15 años,Españoles,Hombres,2020,27979.0
2,2,0-15 años,Españoles,Hombres,2019,28513.0
3,2,0-15 años,Españoles,Mujeres,2021,25547.0
4,2,0-15 años,Españoles,Mujeres,2020,25953.0


Ahora con los de PIB

In [31]:
df_pib = pd.read_csv('..\datos\INE\datos_pib.csv', encoding='latin-1', sep = ';')

Vamos a sustituir las provincias por el nombre y luego mapearemos con el diccionario creado previamente.

In [32]:
df_pib["Provincias"] = df_pib["Provincias"].apply(lambda x:"".join(re.findall(pattern= r" (\D+)", string=x))).map(cod_provincias)

In [33]:
#! Pensándolo ahora se podría simplemente extraer el código de la provincia y listo

Vamos a observar que en los años hay años con `(P)`. Vamos a limpiarlos y pasarlos a int.

In [34]:
df_pib["periodo"].unique()

array(['2021(P)', '2020', '2019'], dtype=object)

In [35]:
df_pib["periodo"] = df_pib["periodo"].str.replace('(P)', "").apply(int)

In [36]:
df_pib["periodo"].unique()

array([2021, 2020, 2019], dtype=int64)

Limpiamos los valores del total.

In [37]:
df_pib["Total"] = df_pib["Total"].apply(decimalpoint)

Vamos a eliminar también las filas que representan totales.

In [38]:
df_pib.drop(index= df_pib[df_pib["Ramas de actividad"] == 'PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO'].index, inplace=True)

In [39]:
df_pib.reset_index(drop=True, inplace=True)

In [40]:
df_pib.head()

,Provincias,Ramas de actividad,periodo,Total
0,2,"A. Agricultura, ganadería, silvicultura y pesca",2021,884324.0
1,2,"A. Agricultura, ganadería, silvicultura y pesca",2020,768976.0
2,2,"A. Agricultura, ganadería, silvicultura y pesca",2019,791464.0
3,2,"B_E. Industrias extractivas, industria manufac...",2021,1397006.0
4,2,"B_E. Industrias extractivas, industria manufac...",2020,1194438.0


In [43]:
df_demog["Edad (3 grupos de edad)"].unique()

array(['0-15 años', '16-64 años', '65 y más'], dtype=object)

In [42]:
ruta_limpios = os.path.join('datos', 'DatosLimpios')
os.makedirs(ruta_limpios, exist_ok=True)

df_demanda.to_csv(os.path.join(ruta_limpios, 'demanda.csv'))
df_generacion.to_csv(os.path.join(ruta_limpios, 'generacion.csv'))
df_demog.to_csv(os.path.join(ruta_limpios, 'demografico.csv'))
df_pib.to_csv(os.path.join(ruta_limpios, 'PIB.csv'))